# Working Notebook for creating wndchrm features

## Step 1
### Load nessecary modules

In [7]:
# Add python modules

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from copy import deepcopy

# Add parent directory for custom modules
from sys import path as sysPath
sysPath.append('../')
sysPath.append('../Machine_Score/')

# Load custom modules that simr has loaded
import main_SIMR as simr
gm = simr.gm
im = simr.im
ic = simr.ic
fe = simr.fe
ms = simr.ms
sa = simr.sa
dc = simr.ms.dc

simr.test()
gm.test()
im.test()
ic.test()
fe.test()
ms.test()
dc.test()
sa.test()

SIMR: Hi!  You're in Matthew's main program for all things galaxy collisions
GM: Hi!  You're in Matthew's module for generally useful functions and classes
IM: Hi!  You're in Matthew's information module for SPAM
IC: Hi!  You're in Matthew's main code for all things image creation.
FE: Hi!  You're in Matthew's module for extracting feature values from images.
MS: Hi!  You're in Matthew's SIMR module for all things machine scoring images
DC: Hi!  You're in direct_image_compare.py
SA: Hi!  You're in Matthew's Main program for score analysis!


___
## Step 2
### Load Target Info class
Needed for loading the target image and running through models later.

In [2]:
tDir = '../targetDir'
tDir = gm.validPath(tDir)

tInfo = im.target_info_class( targetDir = tDir, printAll=False)
if tInfo.status == False:
    print("WARNING: target info class bad")
else:
    print("Target Good!: %s" % tInfo.get('target_id'))

# Get run info class
rInfo = tInfo.getRunInfo( rID='run_0000' )

if rInfo.status == False:
    print("WARNING")
else:
    print("Run Good!: %s"%rInfo.get('run_id'))

Target Good!: 587722984435351614
Run Good!: run_0000


___
## Step 4: Create a new image parameter


In [3]:
chimeLoc = './../param/chime_group_1.json'
chime_group_1 = gm.readJson( chimeLoc )
#gm.pprint(chime_group_1['chime_1'])

In [ ]:

# Function to collect model wndchrm values and normalize them.
def normalize_target_wndchrm( args, tInfo, normDict ):
    
    printAll = args.printAll
    printBase = args.printBase
    
    if printBase:
        print( "FE: normalize_target_wnchrm." )
        gm.tabprint( "tID: %s" % tInfo.get( 'target_id' ) )
        gm.tabprint( "Normalization Parameters")
        gm.pprint( normDict )


fe.normalize_target_wndchrm = normalize_target_wndchrm
fe.normalize_target_wndchrm( tArgNormTest, tInfo, norm_wndchrm_all_test )

In [ ]:
tInfo.printAll = True
tInfo.gatherRunInfos()
print("Done")